# Direct Backend Interaction

This notebook demonstrates how to interact directly with the backend to generate routes, simulating the 'Generate Route' button click.

In [ ]:

import os
import sys
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString
import math

# Add source directories to path
sys.path.append('/home/isma/Desktop/codigo-fuente/src/core/op_blasting_gui/src/enaex_gui')

import route_gen_logic
import utils


## Select Input Data

Define the directory containing the input GeoJSON files.

In [ ]:

# Set the directory with input files
# You can change this to point to your data directory
input_directory = '/home/isma/Desktop/codigo-fuente/src/core/op_blasting_gui/doc/jupyter/generar_rutas/data' 
print(f"Using input directory: {input_directory}")


## Load Data

Load the GeoJSON files from the directory.

In [ ]:

def load_data(directory):
    try:
        geofence = gpd.read_file(os.path.join(directory, "geofence.geojson"))
        home_pose = gpd.read_file(os.path.join(directory, "home_pose.geojson"))
        streets = gpd.read_file(os.path.join(directory, "streets.geojson"))
        
        # Optional files
        try:
            transit_streets = gpd.read_file(os.path.join(directory, "transit_streets.geojson"))
        except:
            transit_streets = None
            
        try:
            obstacles = gpd.read_file(os.path.join(directory, "obstacles.geojson"))
        except:
            obstacles = None
            
        try:
            high_obstacles = gpd.read_file(os.path.join(directory, "high_obstacles.geojson"))
        except:
            high_obstacles = None
            
        # Try to load holes (usually from .hol or geojson)
        try:
             holes = gpd.read_file(os.path.join(directory, "holes.geojson"))
        except:
             print("No holes.geojson found.")
             holes = None

        return holes, geofence, home_pose, streets, transit_streets, obstacles, high_obstacles
    except Exception as e:
        print(f"Error loading files: {e}")
        return None, None, None, None, None, None, None

holes, geofence, home_pose, streets, transit_streets, obstacles, high_obstacles = load_data(input_directory)

if holes is not None:
    print("Data loaded successfully.")
else:
    print("Failed to load essential data.")


## Generate Routes

Call the backend logic to generate routes.

In [ ]:

if holes is not None:
    # Parameters
    use_obstacles = obstacles is not None
    use_high_obstacles = high_obstacles is not None
    use_transit_streets = transit_streets is not None
    fit_streets_enabled = True
    fit_twice = True

    # Call generation logic
    results = route_gen_logic.generate_routes_logic(
        holes,
        geofence,
        home_pose,
        streets,
        transit_streets,
        obstacles,
        high_obstacles,
        use_obstacles,
        use_high_obstacles,
        use_transit_streets,
        fit_streets_enabled,
        fit_twice
    )

    print("Route generation completed.")
else:
    print("Cannot generate routes without data.")


## Process Output

Save the results to `global_plan.csv` and visualize.

In [ ]:

if 'results' in locals():
    all_together = results['all_together']
    output_csv = 'global_plan.csv'
    all_together.to_csv(output_csv)
    print(f"Saved global plan to {output_csv}")

    # Visualization
    fig, ax = plt.subplots(figsize=(10, 10))
    results['geofence'].boundary.plot(ax=ax, color='black')
    results['streets_fitted'].plot(ax=ax, color='blue')
    results['holes_filtered'].plot(ax=ax, color='red', markersize=5)
    if results['blocked'] is not None:
        gpd.GeoSeries(results['blocked']).plot(ax=ax, color='gray', alpha=0.5)

    plt.title("Generated Routes")
    plt.show()
